<a href="https://colab.research.google.com/github/fvillarino/tp-metodo-ensamble/blob/main/08_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [ ]:
import pandas as pd
import numpy as np
import datetime
#from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model

In [ ]:
from google.colab import drive

folder='drive/MyDrive/Colab Notebooks/Kaggle/Rossman/'
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_feather(folder+'train_normalized_data.fth')
df_test = pd.read_feather(folder+'test_normalized_data.fth')

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [ ]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']
# contin_vars = []

In [ ]:
y_out_columns = ['Sales']

In [ ]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [ ]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [ ]:
X_train.shape, X_val.shape

((814150, 40), (30188, 40))

In [ ]:
log_output = False
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

## Categorical columns transformation

In [ ]:
# categoricals_processing = 'no_categoricals'
#categoricals_processing = 'use_onehotencoding'
categoricals_processing = 'use_embeddings'

In [ ]:
X_train

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,PromoInterval,CompetitionOpenSinceYear,Promo2SinceYear,State,Week,Events,Promo_fw,Promo_bw,StateHoliday_bool_fw,StateHoliday_bool_bw,SchoolHoliday_fw,SchoolHoliday_bw,CompetitionDistance,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Precipitationmm,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday_bool,BeforeStateHoliday_bool,Promo,SchoolHoliday,StateHoliday_bool
30188,0,1,2,5,29,0,24,25,2,0,0,13,0,4,26,20,4,2,0,0,0,0,-0.511043,1.733265,1.509328,1.099074,-0.315778,-0.166351,-2.192275,-1.932272,0.036585,-0.317234,-2.314268,0.950700,0.807334,-0.349784,1.072424,1.273237,-0.466372,-0.177442
30189,1,1,2,5,29,0,24,25,0,0,2,12,2,11,26,20,4,2,0,0,0,0,-0.593412,1.378855,1.233050,0.944269,-0.315778,-0.684665,-1.283627,-1.117041,-1.081931,-0.656379,-0.493858,0.950700,0.807334,-0.029087,1.072424,1.273237,-0.466372,-0.177442
30190,2,1,2,5,29,0,24,25,0,0,2,11,3,6,26,0,4,2,0,0,5,2,1.002195,1.733265,1.371189,0.944269,-0.315778,0.092806,-1.359348,-1.422752,-0.410822,-1.165096,-1.859165,1.479808,0.807334,-0.349784,1.072424,1.273237,2.144211,-0.177442
30191,3,1,2,5,29,0,24,25,2,2,0,14,0,0,26,20,4,2,0,0,0,0,-0.587528,1.260719,1.509328,1.718295,-0.315778,-1.462136,-1.737951,-1.422752,-0.187118,-0.317234,-0.493858,-0.019329,0.807334,-0.029087,1.072424,1.273237,-0.466372,-0.177442
30192,4,1,2,5,29,0,2,25,0,0,0,20,0,9,26,20,4,2,0,0,0,0,2.859029,1.378855,1.509328,1.408685,-0.315778,-0.684665,-1.132186,-1.066089,-0.970080,-0.317234,-0.038755,-0.460252,0.807334,-0.349784,1.072424,1.273237,-0.466372,-0.177442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844333,681,1,0,0,0,1,24,25,1,0,0,11,0,0,0,10,1,0,1,1,4,1,-0.642833,-0.747604,-0.424620,-0.139368,-0.012096,0.870277,0.306506,0.615325,0.372140,1.039345,0.416347,0.421593,-0.440285,-1.183596,1.072424,-0.785400,2.144211,5.635633
844334,732,1,0,0,0,1,24,25,1,1,0,4,0,6,0,10,1,0,1,1,4,1,-0.559288,-0.747604,-0.839037,-0.603784,1.714099,0.092806,0.987991,0.768180,1.826211,2.395924,0.416347,-0.195698,-0.440285,-1.183596,1.072424,-0.785400,2.144211,5.635633
844335,768,1,0,0,0,1,24,5,1,1,2,0,4,6,0,10,1,0,1,1,4,1,-0.561641,-0.747604,-0.839037,-0.603784,1.714099,0.092806,0.987991,0.768180,1.826211,2.395924,0.416347,-0.195698,-0.440285,-1.183596,1.072424,-0.785400,2.144211,5.635633
844336,947,1,0,0,0,1,24,25,1,1,0,0,0,1,0,1,1,0,2,1,4,1,-0.492216,-0.747604,-0.977176,-1.223006,-0.215883,-0.036773,0.457947,0.054854,0.036585,-0.995523,0.416347,-0.548436,-0.440285,-1.183596,1.072424,-0.785400,2.144211,5.635633


In [ ]:
if categoricals_processing == 'use_embeddings':
    embeddings_model = load_model(folder+'embeddings_model.hdf5')
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
    X_val = embeddings_model.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

1284/1284 [==============================] - 8s 6ms/step


In [ ]:
X_train.shape

(814150, 146)

In [ ]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    X_val = ohe.transform(X_val)
    X_test = ohe.transform(X_test)

In [ ]:
X_train.shape

(814150, 146)

In [ ]:
X_val.shape

(30188, 146)

In [ ]:
X_test.shape

(41088, 146)

## XGBoost

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [ ]:
n_estimators=2000
learning_rate=0.25
model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, 
                     objective='reg:squarederror', n_jobs=12, max_depth=8, gamma=0.005)
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.005,
             importance_type='gain', learning_rate=0.25, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=12, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [ ]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }

In [ ]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

[0]	validation_0-rmse:0.834685
Will train until validation_0-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:0.338185
[200]	validation_0-rmse:0.333007
[300]	validation_0-rmse:0.338696
Stopping. Best iteration:
[242]	validation_0-rmse:0.331511



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.005,
             importance_type='gain', learning_rate=0.25, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=12, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [ ]:
score = model.score(X_val, y_val)

In [ ]:
score

0.845965350160118

In [ ]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_val)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [ ]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [ ]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [ ]:
print(score, train_RMSE, val_RMSE)

0.8768031658903396 0.1558973858437227 0.16123555251774463


0.8768031658903396 0.1558973858437227 0.16123555251774463 -> n_estimator = 4000, lr = 0.25, gamma=0.005

0.845965350160118 0.1667512806236896 0.1836922340149614 -> n_estimator = 2000, lr = 0.5, gamma=0.01

0.8768031658903396 0.1558973858437227 0.16123555251774463 -> n_estimator = 2000, lr=0.25, gamma=0.005




In [ ]:
model.

242

# Baseline

In [ ]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [ ]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [ ]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [ ]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [ ]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [ ]:
sample_csv['Sales'] = df_test['Sales']

In [ ]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [ ]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [ ]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_XGBoost_{log_output}-{n_estimators}-{learning_rate}.csv', index=False)